<a href="https://colab.research.google.com/github/Hasib0077/nlp1/blob/main/fine_tune_sst2_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install datasets
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding
from torch.utils.data import DataLoader
raw_dataset=load_dataset("glue","sst2")
checkpoint="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(examples):
  return tokenizer(examples["sentence"],padding=True,truncation=True)


tokenized_dataset=raw_dataset.map(tokenize_function,batched=True)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [38]:
# Check the current column names
print("Before modifications:", tokenized_dataset["train"].column_names)

# Remove unnecessary columns (if they exist)
# Since we're using SST-2, there should be no "sentence" or "idx" columns
# Therefore, this line can be commented out or left as is to ensure no errors occur.
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])

# Rename the label column if necessary
if "label" in tokenized_dataset["train"].column_names:
    tokenized_datasets = tokenized_dataset.rename_column("label", "labels")

# Set the format to PyTorch
tokenized_dataset.set_format("torch")

# Check the updated column names
print("After modifications:", tokenized_dataset["train"].column_names)

# Example of accessing the training set
train_dataset = tokenized_dataset["train"]

Before modifications: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']
After modifications: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']


In [39]:
#Define dataloader
from torch.utils.data import DataLoader
train_dataloader=DataLoader(
    tokenized_dataset["train"],shuffle=True,batch_size=8,collate_fn=data_collator
)
eval_dataloader=DataLoader(
    tokenized_dataset["validation"],batch_size=8,collate_fn=data_collator
)



In [40]:
#let's trun to the model
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)
model.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [41]:
 for batch in train_dataloader:
  outputs=model(**batch)
  print(outputs.loss,outputs.logits.shape)
  break


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`sentence` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [42]:
#optimizer
from transformers import AdamW
optimizer=AdamW(model.parameters(),lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [43]:
from transformers import get_scheduler
num_epochs=3
num_training_steps=num_epochs*len(train_dataloader)
lr_scheduler=get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,

)
num_training_steps

25257

In [44]:
#devices
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [45]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/25257 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`sentence` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [13]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

ModuleNotFoundError: No module named 'evaluate'